This notebook merges all the variables (stored in separate tables) from the American Community Survey (ACS) 5-year estimates for the years 2010-2022.

Raw data path: `GLOB~S/Data/U.S. County Data/county_controls/data/ACSDT5Y_2010_2022/`

In [39]:
import pandas as pd
import os

In [40]:
import chardet
ACS_PATH = "/Users/koacow/BOSTON UNIVERSITY Dropbox/Ngoc Duy Khoa Cao/GLOB~S/Data/U.S. County Data/county_controls/data/ACSDT5Y_2010_2022/raw"
# 1.1 Detect encoding of the file
ENCODING = 'utf-8'  # Default encoding
with open(os.path.join(ACS_PATH, "ACSDT5YB02001_2010_2022/ACSDT5Y2022.B02001-Data.csv"), 'rb') as file:
    result = chardet.detect(file.read())
    ENCODING = result['encoding']
ENCODING

'UTF-8-SIG'

In [41]:
# 1.2 Load a sample file to check the structure
sample_file = os.path.join(ACS_PATH, "ACSDT5YB02001_2010_2022/ACSDT5Y2022.B02001-Data.csv")
sample_df = pd.read_csv(sample_file, encoding=ENCODING)
sample_df

,GEO_ID,NAME,B02001_001E,B02001_001M,B02001_002E,B02001_002M,B02001_003E,B02001_003M,B02001_004E,B02001_004M,...,B02001_006M,B02001_007E,B02001_007M,B02001_008E,B02001_008M,B02001_009E,B02001_009M,B02001_010E,B02001_010M,Unnamed: 22
0,Geography,Geographic Area Name,Estimate!!Total:,Margin of Error!!Total:,Estimate!!Total:!!White alone,Margin of Error!!Total:!!White alone,Estimate!!Total:!!Black or African American alone,Margin of Error!!Total:!!Black or African Amer...,Estimate!!Total:!!American Indian and Alaska N...,Margin of Error!!Total:!!American Indian and A...,...,Margin of Error!!Total:!!Native Hawaiian and O...,Estimate!!Total:!!Some Other Race alone,Margin of Error!!Total:!!Some Other Race alone,Estimate!!Total:!!Two or More Races:,Margin of Error!!Total:!!Two or More Races:,Estimate!!Total:!!Two or More Races:!!Two race...,Margin of Error!!Total:!!Two or More Races:!!T...,Estimate!!Total:!!Two or More Races:!!Two race...,Margin of Error!!Total:!!Two or More Races:!!T...,NaN
1,0500000US01001,"Autauga County, Alabama",58761,*****,43747,345,11496,525,59,71,...,30,321,243,2480,668,900,372,1580,533,NaN
2,0500000US01003,"Baldwin County, Alabama",233420,*****,195998,1247,19445,746,848,325,...,32,4414,1216,10638,1169,3952,864,6686,874,NaN
3,0500000US01005,"Barbour County, Alabama",24877,*****,11309,221,11668,220,74,39,...,4,1088,270,609,215,195,97,414,174,NaN
4,0500000US01007,"Bibb County, Alabama",22251,*****,16872,225,4603,294,21,25,...,24,108,129,578,321,109,100,469,310,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3218,0500000US72145,"Vega Baja Municipio, Puerto Rico",54182,*****,21954,1708,2444,629,0,35,...,35,9811,1288,19964,1681,19662,1710,302,147,NaN
3219,0500000US72147,"Vieques Municipio, Puerto Rico",8199,*****,1813,622,579,362,0,21,...,21,5234,700,557,359,549,353,8,18,NaN
3220,0500000US72149,"Villalba Municipio, Puerto Rico",21984,*****,8732,1041,1889,553,16,25,...,28,2868,680,8479,1140,4846,944,3633,764,NaN
3221,0500000US72151,"Yabucoa Municipio, Puerto Rico",30313,*****,2393,494,11349,1294,0,31,...,31,15593,1301,968,393,694,347,274,168,NaN


In [42]:
B02001_dtypes = {
    "GEO_ID": "string",
    "NAME": "string",
    "B02001_001E": "Int64",
    "B02001_002E": "Int64",  
    "B02001_003E": "Int64",  
    "B02001_004E": "Int64",  
    "B02001_005E": "Int64",  
    "B02001_006E": "Int64", 
    "B02001_007E": "Int64",  
    "B02001_008E": "Int64",  
    "B02001_009E": "Int64", 
    "B02001_010E": "Int64",  
}
B02001_FULL = None
# 2.1 Load all B02001 files and concatenate them into a single DataFrame
for year in range(2010, 2023):
    file_path = os.path.join(ACS_PATH, f"ACSDT5YB02001_2010_2022/ACSDT5Y{year}.B02001-Data.csv")
    if os.path.exists(file_path):
        df = pd.read_csv(file_path, encoding=ENCODING, dtype=B02001_dtypes, skiprows=lambda x: x == 1)
        df = df.loc[1:, :] 
        df['year'] = year
        if B02001_FULL is None:
            B02001_FULL = df
        else:
            B02001_FULL = pd.concat([B02001_FULL, df], ignore_index=True)

B02001_FULL = B02001_FULL.drop(columns=['Unnamed: 22'])
B02001_FULL = B02001_FULL.rename(columns={
    "B02001_002E": "white_pop",
    "B02001_003E": "black_pop",
    "B02001_005E": "asian_pop",
    "GEO_ID": "FIPS5",
})
B02001_FULL = B02001_FULL[['FIPS5', 'year', 'white_pop', 'black_pop', 'asian_pop']]
B02001_FULL

,FIPS5,year,white_pop,black_pop,asian_pop
0,0500000US01003,2010,151453,16613,1149
1,0500000US01005,2010,13759,12911,46
2,0500000US01007,2010,19038,3450,36
3,0500000US01009,2010,53938,716,246
4,0500000US01011,2010,2589,8110,47
...,...,...,...,...,...
41850,0500000US72145,2022,21954,2444,9
41851,0500000US72147,2022,1813,579,16
41852,0500000US72149,2022,8732,1889,0
41853,0500000US72151,2022,2393,11349,10


In [43]:
B02001_FULL.isna().sum()

FIPS5        0
year         0
white_pop    0
black_pop    0
asian_pop    0
dtype: int64

In [44]:
B02001_FULL['year'].value_counts()

year
2022    3221
2010    3220
2011    3220
2012    3220
2013    3220
2020    3220
2021    3220
2014    3219
2015    3219
2016    3219
2017    3219
2018    3219
2019    3219
Name: count, dtype: int64

In [45]:
# 2.2 Load all B19013 files and concatenate them into a single DataFrame
B19013_FULL = None
for year in range(2010, 2023):
    file_path = os.path.join(ACS_PATH, f"ACSDT5YB19013_2010_2022/ACSDT5Y{year}.B19013-Data.csv")
    if os.path.exists(file_path):
        df = pd.read_csv(file_path, encoding=ENCODING, skiprows=lambda x: x == 1)
        df['year'] = year
        if B19013_FULL is None:
            B19013_FULL = df
        else:
            B19013_FULL = pd.concat([B19013_FULL, df], ignore_index=True)
B19013_FULL = B19013_FULL.drop(columns=['Unnamed: 4'])
B19013_FULL = B19013_FULL.rename(columns={
    "B19013_001E": "median_household_income",
    "GEO_ID": "FIPS5"
})
B19013_FULL = B19013_FULL[['FIPS5', 'year', 'median_household_income']]
B19013_FULL


,FIPS5,year,median_household_income
0,0500000US01001,2010,53255.0
1,0500000US01003,2010,50147.0
2,0500000US01005,2010,33219.0
3,0500000US01007,2010,41770.0
4,0500000US01009,2010,45549.0
...,...,...,...
41863,0500000US72145,2022,23701
41864,0500000US72147,2022,17062
41865,0500000US72149,2022,22461
41866,0500000US72151,2022,19972


In [46]:
B19013_FULL['year'].value_counts()

year
2022    3222
2010    3221
2011    3221
2012    3221
2013    3221
2020    3221
2021    3221
2014    3220
2015    3220
2016    3220
2017    3220
2018    3220
2019    3220
Name: count, dtype: int64

In [47]:
B19013_FULL.isna().sum()

FIPS5                      0
year                       0
median_household_income    2
dtype: int64

In [48]:
# 2.3 Load all B01003 files and concatenate them into a single DataFrame
B01003_FULL = None
for year in range(2010, 2023):
    file_path = os.path.join(ACS_PATH, f"ACSDT5YB01003_2010_2022/ACSDT5Y{year}.B01003-Data.csv")
    if os.path.exists(file_path):
        df = pd.read_csv(file_path, encoding=ENCODING, dtype={
            "B01003_001E": "Int64",
        }, skiprows=lambda x: x == 1)
        df = df.loc[1:, :]
        df['year'] = year
        if B01003_FULL is None:
            B01003_FULL = df
        else:
            B01003_FULL = pd.concat([B01003_FULL, df], ignore_index=True)
B01003_FULL = B01003_FULL.drop(columns=['Unnamed: 4'])
B01003_FULL = B01003_FULL.rename(columns={
    "B01003_001E": "total_pop",
    "GEO_ID": "FIPS5"
})
B01003_FULL = B01003_FULL[['FIPS5', 'year', 'total_pop']]
B01003_FULL

,FIPS5,year,total_pop
0,0500000US01003,2010,175791
1,0500000US01005,2010,27699
2,0500000US01007,2010,22610
3,0500000US01009,2010,56692
4,0500000US01011,2010,10923
...,...,...,...
41850,0500000US72145,2022,54182
41851,0500000US72147,2022,8199
41852,0500000US72149,2022,21984
41853,0500000US72151,2022,30313


In [49]:
B01003_FULL['year'].value_counts()

year
2022    3221
2010    3220
2011    3220
2012    3220
2013    3220
2020    3220
2021    3220
2014    3219
2015    3219
2016    3219
2017    3219
2018    3219
2019    3219
Name: count, dtype: int64

In [50]:
# 3.1 Load all B15003 files and concatenate them into a single DataFrame
B15003_FULL = None
for year in range(2012, 2023):
    file_path = os.path.join(ACS_PATH, f"ACSDT5YB15003_2012_2022/ACSDT5Y{year}.B15003-Data.csv")
    if os.path.exists(file_path):
        df = pd.read_csv(file_path, encoding=ENCODING, skiprows=lambda x: x == 1)
        df['year'] = year
        if B15003_FULL is None:
            B15003_FULL = df
        else:
            B15003_FULL = pd.concat([B15003_FULL, df], ignore_index=True)

B15003_FULL = B15003_FULL.rename(columns={
    "B15003_022E": "bachelors",
    "B15003_023E": "masters",
    "B15003_024E": "professional",
    "B15003_025E": "doctorate",
    "GEO_ID": "FIPS5",
})
B15003_FULL = B15003_FULL[['FIPS5', 'year', 'bachelors', 'masters', 'professional', 'doctorate']]
B15003_FULL


,FIPS5,year,bachelors,masters,professional,doctorate
0,0500000US01001,2012,5085,1878,415,251
1,0500000US01003,2012,23840,7880,2124,1463
2,0500000US01005,2012,1613,839,224,81
3,0500000US01007,2012,928,335,84,49
4,0500000US01009,2012,3137,1210,334,111
...,...,...,...,...,...,...
35421,0500000US72145,2022,7638,1893,299,171
35422,0500000US72147,2022,406,153,0,316
35423,0500000US72149,2022,2742,466,60,90
35424,0500000US72151,2022,3361,634,102,50


In [51]:
# 4.1 Merge the three DataFrames on 'FIPS5' and 'year'
ACSDT5Y_FULL = B02001_FULL.merge(B19013_FULL, on=['FIPS5', 'year'], how='outer', suffixes=(None, '_y'))
ACSDT5Y_FULL = ACSDT5Y_FULL.drop(columns=[col for col in ACSDT5Y_FULL.columns if col.endswith('_y')])
ACSDT5Y_FULL = ACSDT5Y_FULL.merge(B01003_FULL, on=['FIPS5', 'year'], how='outer', suffixes=(None, '_y'))
ACSDT5Y_FULL =  ACSDT5Y_FULL.drop(columns=[col for col in ACSDT5Y_FULL.columns if col.endswith('_y')])
ACSDT5Y_FULL = ACSDT5Y_FULL.merge(B15003_FULL, on=['FIPS5', 'year'], how='outer', suffixes=(None, '_y'))
ACSDT5Y_FULL = ACSDT5Y_FULL.drop(columns=[col for col in ACSDT5Y_FULL.columns if col.endswith('_y')])

# 4.2 Calculate the 'other_pop' column
ACSDT5Y_FULL['other_pop'] = ACSDT5Y_FULL['total_pop'] - (ACSDT5Y_FULL['white_pop'] + ACSDT5Y_FULL['black_pop'] + ACSDT5Y_FULL['asian_pop'])
ACSDT5Y_FULL

# 4.3 Calculate the 'bachelors_pop' column
ACSDT5Y_FULL['bachelors_pop'] = ACSDT5Y_FULL['bachelors'] + ACSDT5Y_FULL['masters'] + ACSDT5Y_FULL['professional'] + ACSDT5Y_FULL['doctorate']
ACSDT5Y_FULL = ACSDT5Y_FULL.drop(columns=['bachelors', 'masters', 'professional', 'doctorate'])
ACSDT5Y_FULL

,FIPS5,year,white_pop,black_pop,asian_pop,median_household_income,total_pop,other_pop,bachelors_pop
0,0500000US01001,2010,<NA>,<NA>,<NA>,53255.0,<NA>,<NA>,NaN
1,0500000US01001,2011,<NA>,<NA>,<NA>,53899.0,<NA>,<NA>,NaN
2,0500000US01001,2012,<NA>,<NA>,<NA>,53773.0,<NA>,<NA>,7629.0
3,0500000US01001,2013,<NA>,<NA>,<NA>,53682.0,<NA>,<NA>,7472.0
4,0500000US01001,2014,<NA>,<NA>,<NA>,52475.0,<NA>,<NA>,7950.0
...,...,...,...,...,...,...,...,...,...
41863,0500000US72153,2018,28189,1300,5,14954.0,36439,6945,6386.0
41864,0500000US72153,2019,26607,1346,6,14743.0,35428,7469,6304.0
41865,0500000US72153,2020,24931,1244,5,14813,34501,8321,6156.0
41866,0500000US72153,2021,24533,1282,0,16444,34704,8889,6651.0


In [52]:
# 3.3 Keep only the last 5 characters of the FIPS5 code
ACSDT5Y_FULL['FIPS5'] = ACSDT5Y_FULL['FIPS5'].str[-5:]
ACSDT5Y_FULL

,FIPS5,year,white_pop,black_pop,asian_pop,median_household_income,total_pop,other_pop,bachelors_pop
0,01001,2010,<NA>,<NA>,<NA>,53255.0,<NA>,<NA>,NaN
1,01001,2011,<NA>,<NA>,<NA>,53899.0,<NA>,<NA>,NaN
2,01001,2012,<NA>,<NA>,<NA>,53773.0,<NA>,<NA>,7629.0
3,01001,2013,<NA>,<NA>,<NA>,53682.0,<NA>,<NA>,7472.0
4,01001,2014,<NA>,<NA>,<NA>,52475.0,<NA>,<NA>,7950.0
...,...,...,...,...,...,...,...,...,...
41863,72153,2018,28189,1300,5,14954.0,36439,6945,6386.0
41864,72153,2019,26607,1346,6,14743.0,35428,7469,6304.0
41865,72153,2020,24931,1244,5,14813,34501,8321,6156.0
41866,72153,2021,24533,1282,0,16444,34704,8889,6651.0


In [53]:
ACSDT5Y_FULL['year'].value_counts()

year
2022    3222
2010    3221
2011    3221
2012    3221
2013    3221
2020    3221
2021    3221
2014    3220
2015    3220
2016    3220
2017    3220
2018    3220
2019    3220
Name: count, dtype: int64

In [54]:
ACSDT5Y_FULL.isna().sum()

FIPS5                         0
year                          0
white_pop                    13
black_pop                    13
asian_pop                    13
median_household_income       2
total_pop                    13
other_pop                    13
bachelors_pop              6442
dtype: int64

In [55]:
ACSDT5Y_FULL[ACSDT5Y_FULL["FIPS5"] == "06037"]  # Example for Los Angeles County in 2020

,FIPS5,year,white_pop,black_pop,asian_pop,median_household_income,total_pop,other_pop,bachelors_pop
2655,06037,2010,4961910,852780,1340074,55476.0,9758256,2603492,NaN
2656,06037,2011,5126367,844048,1347782,56266.0,9787747,2469550,NaN
2657,06037,2012,5229697,839837,1360561,56241.0,9840024,2409929,1879673.0
2658,06037,2013,5277461,833477,1372726,55909.0,9893481,2409817,1916280.0
2659,06037,2014,5329333,832253,1394349,55870.0,9974203,2418268,1961263.0
2660,06037,2015,5346316,830791,1418362,56196.0,10038388,2442919,2013934.0
2661,06037,2016,5283457,831313,1431361,57952.0,10057155,2511024,2064911.0
2662,06037,2017,5232835,828981,1460508,61015.0,10105722,2583398,2124306.0
2663,06037,2018,5186859,823987,1469968,64251.0,10098052,2617238,2177481.0
2664,06037,2019,5168443,820478,1473221,68044.0,10081570,2619428,2241079.0


In [56]:
OUTPATH = "/Users/koacow/BOSTON UNIVERSITY Dropbox/Ngoc Duy Khoa Cao/GLOB~S/Data/U.S. County Data/county_controls/data/ACSDT5Y_2010_2022/acs_2010_2022.csv"
ACSDT5Y_FULL.to_csv(OUTPATH, index=False, encoding=ENCODING)